<a href="https://colab.research.google.com/github/aimanh22/706_project/blob/main/GPU_Example_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [2]:
#@title Install
!pip install numba
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
!nvidia-smi

/usr/local/cuda-10.1/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so
Sat Oct 24 04:34:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |    111MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+---------

Headers

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import math
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8, float32,float64
from pylab import imshow, show
from timeit import default_timer as timer
import nltk
import pandas as pd
from nltk.corpus import stopwords

In [4]:
print(cuda.gpus)

<Managed Device 0>


In [5]:
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

porter    = nltk.PorterStemmer()

Files

In [7]:
import glob

# read in reviews from the NLTK corpora directory
datadir = '/root/nltk_data/corpora/movie_reviews'
neg     = datadir + '/neg/'
pos     = datadir + '/pos/'

posarr  = []
negarr  = []

pfiles = [f for f in glob.glob(pos + "*.txt", recursive=False)]
nfiles = [f for f in glob.glob(neg + "*.txt", recursive=False)]
for f in pfiles:
    with open(f) as my_file:
        posarr.append(my_file.read().replace('\n',''))
for f in nfiles:
    with open(f) as my_file:
        negarr.append(my_file.read().replace('\n',''))

In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
document_list=posarr+negarr
for i in range(len(document_list)):
    #Converting the words into lower case
    document_list[i]=[w.lower() for w in document_list[i]]
    #Removing the stopwords
    document_list[i]=[w for w in document_list[i] if not str(w) in stop_words]
    #Retain only alphabet and remove everything else- punctuation and numbers
    document_list[i]=[w for w in document_list[i] if w.isalpha()]
    pass
vocab = set()
document_list=[[porter.stem(w) for w in word_list] for word_list in document_list]
#List conatining the number of words in each document 
sizearr={ind:len(list(set(document_list[ind]))) for ind in range(len(document_list))}
#Setting Vocabulary as all unique words in the list
vocab=set(sum(document_list,[]))
# your code for creating the dataframe goes here.
# the resulting dataframe should be called "df"
#
# Each row of the dataframe should represent a document (inaugural address)
# Each column of the dataframe should be a term from the vocab

df = pd.DataFrame()
for doc in document_list: #For each document
        dict_word_count_document=dict() #create an empty dict
        for word in vocab:
            dict_word_count_document[word]=doc.count(word) #update dict with word: count(word)
            pass
        df=df.append(dict_word_count_document,ignore_index=True) #update dataframe
        pass


In [9]:
def computetf(wordvec,doclen):
    tf=dict() #create an empty dict
    for word in wordvec.index:
        tf[word]=wordvec[word]/doclen #append termfreq of word (in the document) in the dict
        pass
    return tf
def computeidf(df):
    idf=dict() #create empty dict
    for word in vocab:
        number_of_docs_with_word=len(df[df[word]>0])
        total_number_of_docs=len(df)
        idf[word]=math.log(total_number_of_docs/number_of_docs_with_word)#append inversetermfreq of the docs
        pass
    return idf        

In [10]:
newdf = pd.DataFrame()

idfdict = computeidf(df)

cols = df.columns
for index, row in df.iterrows():
    newrow = computetf(row,sizearr[index])
    for c in cols:
        newrow[c] = newrow[c]*idfdict[c]
    newdf = newdf.append(newrow,ignore_index=True)

In [11]:
newdf

,b,c,e,f,g,h,j,k,l,n,p,q,r,u,v,w,x,z
0,0.001111,0.002001,0.0,0.0,0.0,0.0,0.007833,0.000834,0.0,0.0,0.001473,0.016564,0.0,0.0,0.000639,0.0,0.003614,0.031055
1,0.000971,0.001236,0.0,0.0,0.0,0.0,0.005805,0.000382,0.0,0.0,0.000883,0.000000,0.0,0.0,0.000324,0.0,0.003826,0.010961
2,0.000765,0.001971,0.0,0.0,0.0,0.0,0.001659,0.000647,0.0,0.0,0.001324,0.008769,0.0,0.0,0.000706,0.0,0.009566,0.000000
3,0.001250,0.003779,0.0,0.0,0.0,0.0,0.010966,0.000750,0.0,0.0,0.002501,0.016564,0.0,0.0,0.001139,0.0,0.016262,0.020703
4,0.001056,0.002751,0.0,0.0,0.0,0.0,0.005483,0.000500,0.0,0.0,0.001834,0.024847,0.0,0.0,0.001223,0.0,0.012648,0.041407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000677,0.001147,0.0,0.0,0.0,0.0,0.002488,0.000382,0.0,0.0,0.000883,0.000000,0.0,0.0,0.000530,0.0,0.003826,0.065763
1996,0.000706,0.001265,0.0,0.0,0.0,0.0,0.000829,0.000412,0.0,0.0,0.000853,0.008769,0.0,0.0,0.000588,0.0,0.000000,0.021921
1997,0.000618,0.002118,0.0,0.0,0.0,0.0,0.002488,0.000324,0.0,0.0,0.001089,0.000000,0.0,0.0,0.000530,0.0,0.005739,0.021921
1998,0.001612,0.003140,0.0,0.0,0.0,0.0,0.006266,0.000778,0.0,0.0,0.002139,0.016564,0.0,0.0,0.001306,0.0,0.009034,0.010352


In [20]:
from numba import (cuda, vectorize,guvectorize)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from functools import wraps
from time import time

def doc_array():
    rea = newdf.to_numpy()
    return np.float32(rea)


@vectorize(['float32(float32, float32)'], target='cuda')
def mul_ufunc(x, y):
    return x*y

@guvectorize(['(float32[:],float64)'],'(n)->()', target='cuda')
def add_ufunc(x,res):
    i=0
    for a in x:
      i=i+a
    res=i

@guvectorize(['(float32[:],float64)'],'(n)->()', target='cuda')
def cal_ufunc(x,res):
    i=0
    for a in x:
      i=i+a*a
    res=i

@vectorize(['float64(float64, float64,float64)'], target='cuda')
def dist_ufunc(d, x, y):
    if x!=0 and y!=0:
      return d/x*y
    else:
      return 0


def cuda_operation():
    """Performs Vectorized Operations on GPU"""
    x=doc_array()
    y=doc_array()
    
    print("Moving calculations to GPU memory")
    x_device = cuda.to_device(x)
    y_device = cuda.to_device(y)
    out_device = cuda.device_array(
        shape=(x_device.shape[0],x_device.shape[1]), dtype=np.float32)
    print(x_device)
    print(x_device.shape)
    print(x_device.dtype)

    print("Calculating on GPU")
    mul_ufunc(x_device,y_device, out=out_device)
    out_host = out_device.copy_to_host()
    mul_val=out_host
    print(mul_val)
    
    mul_device=cuda.to_device(mul_val)
    out_device2 = cuda.device_array(
        shape=(mul_device.shape[0],), dtype=np.float64)
    print(mul_device)
    print(mul_device.shape)
    print(mul_device.dtype)
    print(out_device2.shape)
    add_ufunc(mul_device,out=out_device2)
    a = out_device2.copy_to_host()
    print(f"Calculations from GPU {a}")

    cal_ufunc(x_device,out=out_device2)
    x = out_device2.copy_to_host()
     
    cal_ufunc(y_device,out=out_device2)
    y = out_device2.copy_to_host()

    print(x) 
    print(y)
    
    d=dist_ufunc(x,y,a)
    print(d)
cuda_operation()

Moving calculations to GPU memory
(2000, 18)
float32
Calculating on GPU
[[1.2351856e-06 4.0020013e-06 0.0000000e+00 ... 0.0000000e+00
  1.3058741e-05 9.6440862e-04]
 [9.4251277e-07 1.5267148e-06 0.0000000e+00 ... 0.0000000e+00
  1.4640249e-05 1.2013394e-04]
 [5.8506765e-07 3.8851608e-06 0.0000000e+00 ... 0.0000000e+00
  9.1501563e-05 0.0000000e+00]
 ...
 [3.8167869e-07 4.4866720e-06 0.0000000e+00 ... 0.0000000e+00
  3.2940559e-05 4.8053576e-04]
 [2.5969775e-06 9.8575529e-06 0.0000000e+00 ... 0.0000000e+00
  8.1617130e-05 1.0715651e-04]
 [3.1243994e-07 1.6002812e-06 0.0000000e+00 ... 0.0000000e+00
  3.2940559e-05 4.8053576e-04]]
(2000, 18)
float32
(2000,)
Calculations from GPU [1.97538853e-24 0.00000000e+00 0.00000000e+00 ... 2.84240564e-13
 2.27910401e-23 0.00000000e+00]
[1.97538853e-24 0.00000000e+00 0.00000000e+00 ... 2.84240564e-13
 2.27910401e-23 0.00000000e+00]
[1.97538853e-24 0.00000000e+00 0.00000000e+00 ... 2.84240564e-13
 2.27910401e-23 0.00000000e+00]
[1.97538853e-24 0.000000